In [1]:
import wbgapi as wb
import warnings
import time

import pandas as pd
from pandas import DataFrame
import numpy as np
from numpy import quantile, where
import scipy.stats as stats
import statsmodels as sm
import statsmodels.api
import statsmodels

import pytz
import matplotlib.pyplot as plt
import matplotlib.font_manager
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go

from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
from sklearn.cluster import OPTICS
from sklearn.linear_model import LinearRegression, RANSACRegressor
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import SGDOneClassSVM
from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import make_pipeline
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

warnings.filterwarnings('ignore')

<h1 style="color:black; background-color:white; padding:10px; padding-bottom:10px;text-align: center;">Methodology-for-Outliers-Detection</h1>

<h2 style="color:black; background-color:white; padding:5px; padding-bottom:10px; margin-bottom:-10px">I. Introduction</h2>
<p style="color:black; background-color:white; padding:5px; padding-bottom:20px;margin-bottom:-10px">
Economics as a science of scarcity, places the necessity of studying the economic system in the direction of increasing its efficiency. More and more models close to "reality" are being created that make it possible to understand and predict economic dynamics in order to create effective and responsive economic and social public policies. An important factor in this direction is that all steps from the correct measurement of economic phenomena, to the correct collection of the data, cleaning and preparation of the data are optimal. Only then the models that step on this data will be correct and give the right answers.
<p style="color:black; background-color:white; padding:5px;padding-bottom:20px;margin-bottom:-10px">The present study presents the different methods and techniques for analyzing one of the basic steps before running the models - the detection of outliers and anomalies. The theory of "end values", their types, the factors behind their occurrence and the meaning behind each "end value" will be briefly presented.</p>
<p style="color:black; background-color:white; padding:5px;padding-bottom:20px;margin-bottom:-10px">The purpose of the study is not to show which of these "end values" are bad, wrong or unnecessary, but rather to show different approaches to distinguishing them, which will provide the necessary information for the next step in dealing with them.</p>
<p style="color:black; background-color:white; padding:5px;">As an example of analysis, the database of the World Bank will be used, and specifically indicators for the level of employment and unemployment.</p>

<h2 style="color:black; background-color:white; padding:5px; padding-bottom:10px; margin-bottom:-10px">II. Theoretical foundations of missing data.</h2>
<p style="color:black; background-color:white; padding:5px; margin-bottom:-15px">Let's briefly introduce what outliers and anomalies are in data, how they are related, and how they differ.</p>
<ul style="color:black; background-color:white; padding:5px; margin-bottom:-10px">An outlier is an observation that is unlike the other observations. It is rare, or distinct, or does not fit in some way. Outliers are data points which lie at an abnormal distance from other points in a random sample from a population. In statistics, an outlier is a data point that differs significantly from other observations.</ul>
<li style="color:black; background-color:white; padding:5px"><i>We will generally define outliers as samples that are exceptionally far from the mainstream of the data.</i> — Page 33, Applied Predictive Modeling, 2013.</li>
<li style="color:black; background-color:white; padding:5px"><i>An outlier is an observation that lies outside the overall pattern of a distribution</i> — (Moore and McCabe 1999)</li>
<p style="color:black; background-color:white; padding:5px; padding-bottom:20px; margin-bottom:-15px">Anomalies are referred to values, which do not conform to an expected pattern of the other values in the data set. Anomalies are referred to as a different distribution that occurs within a distribution. In other words, if outliers we can understand them as extreme values, usually single, rare, apparently different caused by multiple factors from errors in collecting part of the data to sharp external influences that distort the values. In contrast, anomalies are rather different values that follow their own behavior, in most cases a few, which is not so much extreme as different in logic from the rest of the values. Reasons that can cause anomalies are the influence of factors at a certain moment that change the behavior of the values. Anomalies are patterns of different data within given data, whereas Outliers would be merely extreme data points within data. If not aggregated appropriately, anomalies may be neglected as outliers. Despite this general definition of outliers and anomalies, in many cases these terms are used interchangeably and the difference between them depending on the case may be difficult to detect. But still revealing the behavior in the final values - random or logical is important when working with them.</p>
<p style="color:black; background-color:white; padding:5px;padding-bottom:20px;margin-bottom:-10px">1. Outliers are divided according to several characteristics. Depending on the number, we distinguish:</p>
<li style="color:black; background-color:white; padding:5px">A univariate outlier is a data point that consists of an extreme value on one variable. </li>
<li style="color:black; background-color:white; padding:5px">A multivariate outlier is a combination of unusual scores on at least two variables. </li>
<p style="color:black; background-color:white; padding:5px;padding-bottom:20px;margin-bottom:-10px">2. Depending on their number and behavior, we distinguish the following three types of outliers:</p>
<li style="color:black; background-color:white; padding:5px">Point outlier: If an individual data point can be considered as anomalous with respect to the rest of the data, then the instance is termed as a point outlier.</li>
<li style="color:black; background-color:white; padding:5px">Contextual outliers: If a data instance is anomalous in a specific context (but not otherwise), then it is termed as a contextual outlier. Attributes of data objects should be divided into two groups :
(Contextual attributes: defines the context, e.g., time & location)
(Behavioural attributes: characteristics of the object, used in outlier evaluation, e.g., temperatur) </li>
<li style="color:black; background-color:white; padding:5px">Collective outliers: If a collection of data points is anomalous concerning the entire data set, it is termed as a collective outlier. </li>
<p style="color:black; background-color:white; padding:5px;padding-bottom:20px;margin-bottom:-10px">3. According to whether we analyze current and future data, we can distinguish the following approaches:</p>
<li style="color:black; background-color:white; padding:5px">outlier detection: The training data contains outliers which are defined as observations that are far from the others. Outlier detection estimators thus try to fit the regions where the training data is the most concentrated, ignoring the deviant observations.</li>
<li style="color:black; background-color:white; padding:5px">novelty detection: The training data is not polluted by outliers and we are interested in detecting whether a new observation is an outlier. In this context an outlier is also called a novelty.</li>
<p style="color:black; background-color:white; padding:5px;padding-bottom:20px;margin-bottom:-10px">Outlier detection and novelty detection are both used for anomaly detection, where one is interested in detecting abnormal or unusual observations. Outlier detection is then also known as unsupervised anomaly detection and novelty detection as semi-supervised anomaly detection.</p>
<p style="color:black; background-color:white; padding:5px;padding-bottom:20px;margin-bottom:-10px">4. Depending on the reasons for the occurrence of outliers, we can designate the most common ones as:</p>
<li style="color:black; background-color:white; padding:5px">Errors in measuring or collecting the data </li>
<li style="color:black; background-color:white; padding:5px">Errors in data extraction, processing or manipulation</li>
<li style="color:black; background-color:white; padding:5px">Human error when entering, loading the data.</li>
<li style="color:black; background-color:white; padding:5px">Natural occurrences of outliers that aren’t errors, which can be called dataset novelties.</li>
<p style="color:black; background-color:white; padding:5px;padding-bottom:20px;margin-bottom:-10px">5. The importance of detecting outliers and anomalies is enormous. Not only the improvement of the data from the point of view of applying subsequent models but also as a red light in many cases. As examples of the importance of detecting outliers and anomalies we can mention:</p>
<li style="color:black; background-color:white; padding:5px">Medicine: An anomalous values in blood or other indicators may indicate the presence of a malignant tumor, or other hidden illnes;</li>
<li style="color:black; background-color:white; padding:5px">Cyberattacks: Outlieres may indicate security breaches;</li>
<li style="color:black; background-color:white; padding:5px">Fraud detection: An outlier credit card transaction or an abnormal buying pattern can indicate a credit card theft or an identity theft;</li>

<h2 style="color:black; background-color:white; padding:5px; padding-bottom:10px; margin-bottom:-10px">III. Methodology and empirical analysis </h2>
<p style="color:black; background-color:white; padding:5px;padding-bottom:20px;margin-bottom:-10px">The methodology of the present study is based on the study of economic data related to the labor market. The aim of the research is to reveal the different techniques for distinguishing anomalies and outliers in economic data. The data parameters are as follows:</p>
<li style="color:black; background-color:white; padding:5px">Database used: World Bank ( https://data.worldbank.org/ ) </li>
<li style="color:black; background-color:white; padding:5px">Indicators used: employment rate (https://data.worldbank.org/indicator/SL.EMP.TOTL.SP.NE.ZS) and unemployment rate (https://data.worldbank.org/indicator/SL. UEM.TOTL.NE.ZS).</li>
<li style="color:black; background-color:white; padding:5px">Year of analysis: 2015</li>
<li style="color:black; background-color:white; padding:5px">Scope of analysis: countries worldwide (WLD)</li>

<p style="color:black; background-color:white; padding:5px;padding-bottom:20px;margin-bottom:-10px">The methodology includes the following two main steps:</p>
<li style="color:black; background-color:white; padding:5px">Preprocessing: preparing the data for analysis including loading the data, familiarizing the data and removing missing data. </li>
<li style="color:black; background-color:white; padding:5px">Anomaly/outlier analysis: using different tools to reveal outliers based on graphical method, statistical toolkit, distance and depth techniques, deep learning methods, clustering. linear etc.</li>

<h3 style="color:black; background-color:white; padding:5px; margin-bottom:-15px">Step one. Preprocessing</h3>
<ul style="color:black; background-color:white; padding:5px; margin-bottom:-10px"> In this first step, the database will be prepared using the following sub-steps:</ul>
<li style="color:black; background-color:white; padding:5px">Data from the World Bank database will be loaded;</li>
<li style="color:black; background-color:white; padding:5px">The key features of the data will be revealed;</li>
<li style="color:black; background-color:white; padding:5px">Missing data will be removed</li>
<li style="color:black; background-color:white; padding:5px">The data will be visualized</li>

<h4 style="color:black; background-color:white; padding:5px; padding-bottom:10px; margin-bottom:-10px">Load the data</h4>
<p style="color:black; background-color:white; padding:5px;">
A function is created to load the data from the World Bank database according to selected parameters: employment and unemployment indicators with a worldwide scope.</p>

In [2]:
def load_and_name_db_WB (db, *args):
    global df_name
    df_name = db
    globals()[df_name] = wb.data.DataFrame(indicators, wb.region.members(region), range(start_period, end_period))
    globals()[df_name].columns = (new_column_names)
    globals()[df_name] = globals()[df_name].round(2) 
    return globals()[df_name]

In [3]:
name_db = "empl_unempl_world"
indicators = ['SL.UEM.TOTL.NE.ZS', "SL.EMP.TOTL.SP.NE.ZS"]
new_column_names = ['employment', "unemployment"]
region = "WLD"
start_period = 2015
end_period = 2016

load_and_name_db_WB(name_db, indicators, region, start_period, end_period)

,employment,unemployment
economy,,
ABW,NaN,NaN
AFG,NaN,NaN
AGO,NaN,NaN
ALB,45.96,17.19
AND,NaN,NaN
...,...,...
XKX,22.49,32.84
YEM,NaN,NaN
ZAF,44.50,22.87


<h4 style="color:black; background-color:white; padding:5px; padding-bottom:10px; margin-bottom:-10px">Extraction of basic information about the data</h4>
<p style="color:black; background-color:white; padding:5px;">
In addition to the previous substep, it is important to outline the database parameters such as size, number of cases, data type, etc. In this way, we will understand what the "idea" of the database is and what can and should be done with it in terms of technical content.</p>

In [4]:
def db_info (df):
    observations = None
    features = None
    observations, features = df.shape
    print("1. Оbservations and features: \n {} Оbservations and {} features".format(observations, features))
    print("------------------------------------------------------")
    print (f"2. Number of cases in the table: {df.size}")
    print("------------------------------------------------------")
    print(f"3. The sum of element types by type is as follows: \n {df.dtypes.value_counts(ascending=True)}")
    print("------------------------------------------------------")

    list_objects = []
    list_int = []
    list_float64 = []

    for col in df.columns:
        if df[col].dtypes == "object":
            list_objects += [col]
        elif df[col].dtypes == "float64":
            list_float64 += [col]
        elif df[col].dtypes == "int64" or df[col].dtypes == "int32":
            list_int += [col]
    print("4. Group the features by data type:")
    print(f" object = {list_objects} \n")
    print(f" int = {list_int} \n")
    print(f" float64 = {list_float64}")

In [5]:
db_info(empl_unempl_world)

1. Оbservations and features: 
 217 Оbservations and 2 features
------------------------------------------------------
2. Number of cases in the table: 434
------------------------------------------------------
3. The sum of element types by type is as follows: 
 float64    2
dtype: int64
------------------------------------------------------
4. Group the features by data type:
 object = [] 

 int = [] 

 float64 = ['employment', 'unemployment']
